## Import Things

In [ ]:
import gpt_2_simple as gpt2
from big_phoney import BigPhoney
import numpy as np
import lyricsgenius as genius
import json
import pandas as pd

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action
# model_name = "124M"
# model is saved into current directory under /models/124M/
# gpt2.download_gpt2(model_name=model_name) #need to run only once. comment out once done.

## Load models and outside functions

In [ ]:
#Initiate TensorFlow session, needs GPU
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

#Load text augmentation model
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert")

#For syllable counting
phoney = BigPhoney()

## Functions 

These will go in a Utils.py file 

I'll also need a get_target_lyrics function so I can get whatever lyrics the user wants as the target. Will do this in another file. 

In [ ]:
def get_target_lyrics():
    api_key = 'Nc3VjqRNlEZiD9mqOQATDQ5PuZ4IjdgV13n7c39OIyNwsjuazZThQlQkdr_Hts4c'
    api = genius.Genius(api_key)
    api.remove_section_headers = True # Removes section headers like "Verse" and "Intro"
    input_artist = ("Idina Menzel")
    input_title = ("Let it go")
    #artist = api.search_artist(input_artist)
    song = api.search_song(input_title, input_artist)
    
    #clean up the target lyrics
    target = song.lyrics
    target = target.replace("\u2005"," ")
    target = target.replace("\\"," ")
    target = target.replace("\\n"," ")
    target = target.replace("("," ")
    target = target.replace(")"," ")
    target = target.replace("\n\n"," ")
    target = target.replace("\n\n\n"," ")
    #create list of lines
    target_lyrics = target.split("\n")
    
    return target_lyrics

In [ ]:
# Later on I'll need to add the artist as a parameter, if I have 3, so it can load in the right finetuned model
def generate_lyrics(): 
    
    generated_text = gpt2.generate(sess, 
                     length=250,
                     temperature=0.9,
                     prefix="Deep in the night",
                     nsamples=1,
                     batch_size=1,
                     return_as_list=True)[0]
    
    generated_text = generated_text.split("\n")
    
    return generated_text, target_lyrics

This next small part seems unnecessary. Should be able to just get generated text and target lyrics straight from function

In [ ]:
lyrics = generate_lyrics()
gen_lyrics = lyrics[0]
target_lyrics = lyrics[1]

Here's something that counts syllables

In [ ]:
def count_syls(text):
    
    gen_schema = []
    
    for line in text:
         syls = phoney.count_syllables(line)
         gen_schema.append(syls)
         #print(syls,line)
        
    return gen_schema

The next two parts also seems like they should be in functions

In [ ]:
gen_schema = count_syls(gen_lyrics)
target_schema = count_syls(target_lyrics)

In [ ]:
target_len = len(target_schema)
del gen_schema[target_len:]
del gen_lyrics[target_len:]
#print(gen_schema, len(gen_schema), len(target_schema))

#### The magical lyrics fitting function that hopefully works consistently!

In [ ]:
def fit_lyrics(gen_schema, gen_lyrics, target_schema, target_lyrics)
    new_lyrics = []
    for num, line in enumerate(gen_lyrics):
        if (gen_schema[num] == target_schema[num]): 
            new_lyrics.append(line)
        elif (gen_schema[num] < target_schema[num]):
            syls = gen_schema[num]
            while syls < target_schema[num]:
                new_line = aug.augment(line)
                syls = phoney.count_syllables(new_line)
                #In case we overshoot (add too many syllables)
                if syls > target_schema[num]:
                    print("Oops we overshot")
                    print(syls)
                    print("For line:")
                    print(num)
                    new_line = line
                #print("---------")
            new_lyrics.append(new_line)
        elif (gen_schema[num] > target_schema[num]):
            syls = gen_schema[num]
            words = line.split(" ")
            while syls > target_schema[num]:
                del words[-1]
                new_line = ' '.join(words)
                syls = phoney.count_syllables(new_line)
                #In case too many syllables are deleted
                if syls < target_schema[num]:
                    print("Oops, deleted too many")
                    new_line = aug.augment(new_line)
                    syls = phoney.count_syllables(new_line)
            new_lyrics.append(new_line)
    print(new_lyrics)
    
    return new_lyrics